In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from neural_network import NeuralNetworkModel, ChessDataset, ChessEvaluationNeuralNetwork

In [ ]:
df = pd.read_csv("../../data/chess_data.csv")

df = df[~df['Evaluation'].str.contains('#', na=False)]
X = df['FEN'].apply(NeuralNetworkModel.fen_to_feature_array).tolist()
X = np.array(X, dtype=np.float32)
y = df['Evaluation'].apply(lambda x: float(x)).astype(np.float32).values.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


train_dataset = ChessDataset(X_train, y_train)
test_dataset = ChessDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

99445


In [9]:
model = ChessEvaluationNeuralNetwork(input_size=X.shape[1])
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(100):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader):.4f}")

Epoch 1, Loss: 366673.3426
Epoch 2, Loss: 344497.0250
Epoch 3, Loss: 323682.0047
Epoch 4, Loss: 312177.1111
Epoch 5, Loss: 300682.6560
Epoch 6, Loss: 290217.8505
Epoch 7, Loss: 283210.7884
Epoch 8, Loss: 275748.6592
Epoch 9, Loss: 269513.0100
Epoch 10, Loss: 263012.3995
Epoch 11, Loss: 257303.1214
Epoch 12, Loss: 249989.6456
Epoch 13, Loss: 247548.1921
Epoch 14, Loss: 239893.4793
Epoch 15, Loss: 234378.9842
Epoch 16, Loss: 232860.3281
Epoch 17, Loss: 228083.5047
Epoch 18, Loss: 221183.7003
Epoch 19, Loss: 219043.0179
Epoch 20, Loss: 210897.2728
Epoch 21, Loss: 205770.7189
Epoch 22, Loss: 203129.9813
Epoch 23, Loss: 199805.3085
Epoch 24, Loss: 195121.6153
Epoch 25, Loss: 193357.4672
Epoch 26, Loss: 186828.1628
Epoch 27, Loss: 183193.0778
Epoch 28, Loss: 181511.5659
Epoch 29, Loss: 179507.5280
Epoch 30, Loss: 172510.3164
Epoch 31, Loss: 166993.4629
Epoch 32, Loss: 167765.3292
Epoch 33, Loss: 165591.6568
Epoch 34, Loss: 162539.7240
Epoch 35, Loss: 160729.1550
Epoch 36, Loss: 155266.0793
E

In [10]:
model.eval()
test_loss = 0.0
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss.item()

avg_test_loss = test_loss / len(test_loader)
rmse_error = np.sqrt(avg_test_loss)
print(f"\nTest RMSE: {rmse_error:.4f}")


Test RMSE: 520.6366


In [14]:
torch.save(model.state_dict(), 'neural_network.pth')

In [ ]:
print("test")